<a href="https://colab.research.google.com/github/AhmedGabl/sentiment-analysis-rnn/blob/main/Sentiment_Analysis_GNasr52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment Analysis using RNN

Text Classification Task: Sentiment Analysis

- Data Preparation: Today we use a dataset (IMDb) for movie reviews. Each review is labeled as either positive or negative.

- Preprocessing: Tokenize the text and convert words to integers. Pad sequences to ensure they have the same length.

- Model Definition:
using an RNN layer to capture the sequential nature of the reviews.
Add a Dense layer with a sigmoid activation for binary classification.

- Training:
Train the model on the training dataset.

- Evaluation:
Test on a separate validation set and evaluate performance using metrics like accuracy or F1-score.

**Problem Statement:**

In this, we have to predict the number of positive and negative reviews based on sentiments by using RNN archticture. This is workable example on Many to One type as it takes sentances and output if it's negative or positive.

In [1]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional

from numpy import asarray
from numpy import zeros

from numpy import asarray
from numpy import zeros
from tensorflow.keras.layers import GRU

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding
from tensorflow.keras.layers import Flatten


import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
!pip install kaggle

In [3]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 20.7MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.1MB/s]


In [4]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [5]:
raw_data = pd.read_csv("IMDB Dataset.csv")
print(raw_data.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [6]:
X = raw_data['review']  # Features: reviews
raw_data['label'] = raw_data['sentiment'].map({'positive': 1, 'negative': 0})
y = raw_data['label']  # Labels: sentiment (positive/negative)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 40000
Testing set size: 10000


### Data Prerocessing Pipeline

- remove_special_chars(text)

Purpose: Clean the input text by removing special characters and HTML entities.

Steps:

Compile a regex to match multiple spaces.

Convert text to lowercase.

Replace specific HTML character codes with their corresponding characters.

Replace newline characters and HTML tags with appropriate representations.

Use html.unescape to convert any remaining HTML entities.

Replace multiple spaces with a single space.

- remove_non_ascii(text)

Purpose: Eliminate non-ASCII characters from the text.

Steps:

Normalize the text to a compatible Unicode format.

Encode to ASCII, ignoring non-ASCII characters.

Decode back to UTF-8 format.

- to_lowercase(text)

Purpose: Convert all characters in the text to lowercase.

Steps:

Simply return the text converted to lowercase.

- remove_punctuation(text)

Purpose: Strip punctuation from the text.

Steps:

Create a translation table that maps punctuation characters to None.

Use the translation table to translate the text.

- replace_numbers(text)

Purpose: Remove all integer occurrences from the text.

Steps:

Use a regex to find and replace all digits with an empty string.

- remove_whitespaces(text)

Purpose: Trim leading and trailing whitespace from the text.

Steps:

Return the text after applying the strip() method.

- remove_stopwords(words, stop_words)

Purpose: Filter out common stopwords from a list of words.

Steps:

Return a list of words that are not present in the provided stop_words set.

- stem_words(words)

Purpose: Apply stemming to a list of words.

Steps:

Create an instance of a stemmer.

Return a list of stemmed words using the stemmer.

- lemmatize_words(words)

Purpose: Lemmatize words in the text to their base form.

Steps:

Create an instance of a lemmatizer.

Return a list of lemmatized words.

- lemmatize_verbs(words)

Purpose: Specifically lemmatize verbs in the text.

Steps:

Create an instance of a lemmatizer.

Return a string of lemmatized verbs, maintaining space between words.

- text2words(text)

Purpose: Tokenize the input text into a list of words.

Steps:

Use a word tokenizer to split the text into individual words and return the list.

In [7]:

def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    # words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [8]:
def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [10]:
import nltk
nltk.download('punkt_tab')
proc_X_train = normalize_corpus(X_train)
proc_X_test = normalize_corpus(X_test)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2025-01-25 20:12:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-01-25 20:12:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-01-25 20:12:03--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [12]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


### Building Pre-trained word embeddings using Glove6B With Bi-Directional LSTM

Overview of Layers

- Embedding Layer:

This layer initializes with the pre-trained GloVe embeddings.

You will need to load the GloVe vectors and create an embedding matrix where each word index corresponds to its GloVe vector.

- Bidirectional LSTM Layer:

This layer processes sequences in both forward and backward directions, capturing context from both sides.
It consists of two LSTM layers: one for the forward pass and another for the backward pass.

- Dense Layer(s):

Typically, you'll have one or more fully connected layers to output your final predictions.
The last dense layer often uses a softmax activation for classification tasks.

- Output Layer:

This layer generates the final predictions, which can be class labels, probabilities, etc.

In [13]:
t = Tokenizer()
t.fit_on_texts(proc_X_train)  # Fit on training data only
vocab_size = len(t.word_index) + 1

# Integer encode the training documents
encoded_train_docs = t.texts_to_sequences(proc_X_train)
# Integer encode the testing documents
encoded_test_docs = t.texts_to_sequences(proc_X_test)

# Pad documents to a max length of 100 words (adjust as necessary)
max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

# Load the whole embedding into memory (make sure to have the GloVe file I added it above when I used stanford api)
embeddings_index = dict()
with open('glove.6B.100d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

# Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define model using LSTM
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True)))  # LSTM layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Bidirectional(LSTM(100)))  # Another LSTM layer
model.add(Dropout(0.5))  # Another Dropout layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Summarize the model
model.summary()

# Fit the model
model.fit(padded_train_docs, y_train, epochs=5, verbose=1)  # Reduced epochs for quicker training

# Evaluate the model on the test set
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy * 100))

Loaded 400000 word vectors.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │      12,297,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,297,500 (46.91 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 12,297,500 (46.91 MB)

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 20ms/step - acc: 0.7346 - loss: 0.5242
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - acc: 0.8343 - loss: 0.3818
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 20ms/step - acc: 0.8515 - loss: 0.3478
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - acc: 0.8731 - loss: 0.3073
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - acc: 0.8794 - loss: 0.2882
Test Accuracy: 86.949998


In [14]:
# Prepare tokenizer
t = Tokenizer()
t.fit_on_texts(proc_X_train)  # Fit on training data only
vocab_size = len(t.word_index) + 1

# Integer encode the training documents
encoded_train_docs = t.texts_to_sequences(proc_X_train)
# Integer encode the testing documents
encoded_test_docs = t.texts_to_sequences(proc_X_test)

# Pad documents to a max length of 100 words (adjust as necessary)
max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

# Load the whole embedding into memory (make sure to have the GloVe file)
embeddings_index = dict()
with open('glove.6B.100d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

# Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define model using Vanilla RNN
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(SimpleRNN(100, return_sequences=False))  # Simple RNN layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Summarize the model
model.summary()

# Fit the model
model.fit(padded_train_docs, y_train, epochs=5, verbose=1)  # Reduced epochs for quicker training

# Evaluate the model on the test set
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy * 100))


Loaded 400000 word vectors.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │      12,297,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,297,500 (46.91 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 12,297,500 (46.91 MB)

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - acc: 0.5300 - loss: 0.7141
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - acc: 0.5651 - loss: 0.6808
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - acc: 0.5349 - loss: 0.6872
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - acc: 0.5267 - loss: 0.6938
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - acc: 0.5394 - loss: 0.6871
Test Accuracy: 59.640002


In [15]:
# Prepare tokenizer
t = Tokenizer()
t.fit_on_texts(proc_X_train)  # Fit on training data only
vocab_size = len(t.word_index) + 1

# Integer encode the training documents
encoded_train_docs = t.texts_to_sequences(proc_X_train)
# Integer encode the testing documents
encoded_test_docs = t.texts_to_sequences(proc_X_test)

# Pad documents to a max length of 100 words (adjust as necessary)
max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

# Load the whole embedding into memory (make sure to have the GloVe file)
embeddings_index = dict()
with open('glove.6B.100d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

# Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define model using GRU
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(GRU(100))  # GRU layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Summarize the model
model.summary()

# Fit the model
model.fit(padded_train_docs, y_train, epochs=5, batch_size = 32, verbose=1)  # Reduced epochs for quicker training

# Evaluate the model on the test set
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy * 100))


Loaded 400000 word vectors.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │      12,297,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,297,500 (46.91 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 12,297,500 (46.91 MB)

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - acc: 0.6716 - loss: 0.5750
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - acc: 0.8472 - loss: 0.3564
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - acc: 0.8631 - loss: 0.3220
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - acc: 0.8758 - loss: 0.2987
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - acc: 0.8918 - loss: 0.2644
Test Accuracy: 86.580002


In [ ]:

import os
# Save the Keras model
model.save('model.keras')

# Create a directory to store necessary files (if it doesn't exist)
if not os.path.exists('deployment_files'):
    os.makedirs('deployment_files')

# Copy the model file into that directory
!cp model.keras deployment_files/

# Save other necessary files (tokenizer, vocabulary, etc.) as needed.
# Example: saving the tokenizer
import pickle
with open('deployment_files/tokenizer.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)
